# Machine Learning Project (Part 1 - ANN Models ) 

## Load And Prepare Images 

In [1]:
# let's turn the data into structured form 
import numpy as np

# import warnings
import warnings
# filter warnings
warnings.filterwarnings('ignore')

#%%Loading data
from os import listdir
import numpy as np 
from PIL import Image

X = []    # X -> to store images as 2D arrays 
Y = []    # Y -> to store one hot encoding 
# ----------------------------------- load images-------------------------------------
# get the image directory

for i in range(10):
    folder_dir = f"/kaggle/input/sign-language-dataset/Sign-Language-Digits-Dataset-master/Dataset/{i}"
    for image in listdir(folder_dir):
 
    # check if the image ends with JPG
        
        if (image.endswith(".JPG")):
            #  Convert each image to gray
            img = Image.open(folder_dir+'/'+image).convert('L')
            # we have to check the image size before loading it 
            width, height = img.size
            if width == 100 and height == 100:
                # GET X,y to be ready 
                X.append(np.array(img))
                Y.append([1 if j==i else 0 for j in range(10)])
            

# turn X to 3D array as it's a list right now 

X = np.array(X)  # contain images in 2D shape
Y = np.array(Y) # contain one hot encoder for images 
# Divide each image by 255
X = X/255.0

# just to make sure let's check the shape of (X,y)
print ("image 3D array shape:",X.shape)
print ("image one hot encoder label:",Y.shape)

# now let's store labels in y 
y = np.argmax(Y,axis=1)
print ("images classes vector: ",y) 


image 3D array shape: (2059, 100, 100)
image one hot encoder label: (2059, 10)
images classes vector:  [0 0 0 ... 9 9 9]


## NN : First Structure Model (Softmax and Sigmoid)

## sigmoid 

In [2]:
# building our model - with 2 hidden layers (sigmoid)
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from keras.models import Sequential # initialize neural network library
from keras.layers import Dense # build our layers library


#building our model
def build_classifier():
    classifier = Sequential() # initialize neural network
    # first Hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train_flatten.shape[1]))
    # Second hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # Output layer: we can use sigmoid function as we deal with [0,1] arrays for Y   
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 200)


#%% Train Test split
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
# Reshape
X_train = X_train.reshape(-1, 100, 100,1)
X_test = X_test.reshape(-1, 100, 100, 1)

#ANN data preprocessing
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]

#  we need to but X_train and X_test in 2D form for NN 
X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
print("X train flatten",X_train_flatten.shape)
print("X test flatten",X_test_flatten.shape)

#%%training and testing

#evaluate and test ANN model
# cross validation and measurement of accuracy
train_accuracies = cross_val_score(estimator = classifier, X = X_train_flatten, y = Y_train, cv = 3)
test_accuracies = cross_val_score(estimator = classifier, X = X_test_flatten, y = Y_test, cv = 3)

train_mean = train_accuracies.mean()
test_mean = test_accuracies.mean()

print ("First Model results - sigmoid")
print("ANN train accuracy mean: "+ str(train_mean))
print("ANN test accuracy mean: "+ str(test_mean))

Using TensorFlow backend.


X train flatten (1647, 10000)
X test flatten (412, 10000)
Epoch 1/200
1098/1098 [==============================] - 1s 532us/step - loss: 0.4523 - accuracy: 0.8472
Epoch 2/200
1098/1098 [==============================] - 0s 241us/step - loss: 0.3270 - accuracy: 0.9000
Epoch 3/200
1098/1098 [==============================] - 0s 195us/step - loss: 0.3261 - accuracy: 0.9000
Epoch 4/200
1098/1098 [==============================] - 0s 166us/step - loss: 0.3265 - accuracy: 0.9000
Epoch 5/200
1098/1098 [==============================] - 0s 159us/step - loss: 0.3263 - accuracy: 0.9000
Epoch 6/200
1098/1098 [==============================] - 0s 177us/step - loss: 0.3262 - accuracy: 0.9000
Epoch 7/200
1098/1098 [==============================] - 0s 175us/step - loss: 0.3261 - accuracy: 0.9000
Epoch 8/200
1098/1098 [==============================] - 0s 177us/step - loss: 0.3268 - accuracy: 0.9000
Epoch 9/200
1098/1098 [==============================] - 0s 175us/step - loss: 0.3260 - accuracy: 0.90

## Softmax 

In [3]:
#building our model - softmax 
def build_classifier():
    classifier = Sequential() # initialize neural network
    # first Hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train_flatten.shape[1]))
    # Second hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # Output layer: we will use softmax function  
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 200)


#%% Train Test split - we will use labeled y with classes using argmax 
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size)
# Reshape
X_train = X_train.reshape(-1, 100, 100,1)
X_test = X_test.reshape(-1, 100, 100, 1)

#ANN data preprocessing
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]

#  we need to but X_train and X_test in 2D form for NN 
X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
print("X train flatten",X_train_flatten.shape)
print("X test flatten",X_test_flatten.shape)

#%%training and testing

#evaluate and test ANN model
# cross validation and measurement of accuracy
train_accuracies = cross_val_score(estimator = classifier, X = X_train_flatten, y = Y_train, cv = 3)
test_accuracies = cross_val_score(estimator = classifier, X = X_test_flatten, y = Y_test, cv = 3)

train_mean = train_accuracies.mean()
test_mean = test_accuracies.mean()

print ("First Model results - Softmax")
print("ANN train accuracy mean: "+ str(train_mean))
print("ANN test accuracy mean: "+ str(test_mean))

X train flatten (1647, 10000)
X test flatten (412, 10000)
Epoch 1/200
1098/1098 [==============================] - 0s 335us/step - loss: 2.3032 - accuracy: 0.1047
Epoch 2/200
1098/1098 [==============================] - 0s 207us/step - loss: 2.3010 - accuracy: 0.1047
Epoch 3/200
1098/1098 [==============================] - 0s 225us/step - loss: 2.2990 - accuracy: 0.1193
Epoch 4/200
1098/1098 [==============================] - 0s 212us/step - loss: 2.2957 - accuracy: 0.1047
Epoch 5/200
1098/1098 [==============================] - 0s 164us/step - loss: 2.2871 - accuracy: 0.1075
Epoch 6/200
1098/1098 [==============================] - 0s 162us/step - loss: 2.2790 - accuracy: 0.1384
Epoch 7/200
1098/1098 [==============================] - 0s 156us/step - loss: 2.2531 - accuracy: 0.1412
Epoch 8/200
1098/1098 [==============================] - 0s 159us/step - loss: 2.2045 - accuracy: 0.1630
Epoch 9/200
1098/1098 [==============================] - 0s 162us/step - loss: 2.1667 - accuracy: 0.17

# NN Second Structure Model (Softmax and Sigmoid)

## Sigmoid

In [11]:
#building our model - with 5 hidden layer 
def build_classifier():
    classifier = Sequential() # initialize neural network
    # first Hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train_flatten.shape[1]))
    # Second hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # third hidden layer with 24 neurons and random uniform values for weights 
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))
    # fourth hidden layer with 24 neurons and random uniform values for weights 
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))
    # fifth hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # Output layer: we can use sigmoid function as we deal with [0,1] arrays for Y   
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'sigmoid'))
    classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 200)


#%% Train Test split
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=test_size)
# Reshape
X_train = X_train.reshape(-1, 100, 100,1)
X_test = X_test.reshape(-1, 100, 100, 1)

#ANN data preprocessing
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]

#  we need to but X_train and X_test in 2D form for NN 
X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
print("X train flatten",X_train_flatten.shape)
print("X test flatten",X_test_flatten.shape)


#evaluate and test ANN model
# cross validation and measurement of accuracy
train_accuracies = cross_val_score(estimator = classifier, X = X_train_flatten, y = Y_train, cv = 3)
test_accuracies = cross_val_score(estimator = classifier, X = X_test_flatten, y = Y_test, cv = 3)

train_mean = train_accuracies.mean()
test_mean = test_accuracies.mean()

print ("Second Model results - Sigmoid")
print("ANN train accuracy mean: "+ str(train_mean))
print("ANN test accuracy mean: "+ str(test_mean))

X train flatten (1647, 10000)
X test flatten (412, 10000)
Epoch 1/200
1098/1098 [==============================] - 1s 552us/step - loss: 0.6730 - accuracy: 0.8914
Epoch 2/200
1098/1098 [==============================] - 0s 229us/step - loss: 0.3688 - accuracy: 0.9000
Epoch 3/200
1098/1098 [==============================] - 0s 189us/step - loss: 0.3269 - accuracy: 0.9000
Epoch 4/200
1098/1098 [==============================] - 0s 156us/step - loss: 0.3262 - accuracy: 0.9000
Epoch 5/200
1098/1098 [==============================] - 0s 155us/step - loss: 0.3263 - accuracy: 0.9000
Epoch 6/200
1098/1098 [==============================] - 0s 158us/step - loss: 0.3263 - accuracy: 0.9000
Epoch 7/200
1098/1098 [==============================] - 0s 162us/step - loss: 0.3263 - accuracy: 0.9000
Epoch 8/200
1098/1098 [==============================] - 0s 165us/step - loss: 0.3262 - accuracy: 0.9000
Epoch 9/200
1098/1098 [==============================] - 0s 145us/step - loss: 0.3260 - accuracy: 0.90

## Softmax

In [7]:
#building our model - with 5 hidden layer 
def build_classifier():
    classifier = Sequential() # initialize neural network
    # first Hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 8, kernel_initializer = 'uniform', activation = 'relu', input_dim = X_train_flatten.shape[1]))
    # Second hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # third hidden layer with 24 neurons and random uniform values for weights 
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))
    # fourth hidden layer with 24 neurons and random uniform values for weights 
    classifier.add(Dense(units = 24, kernel_initializer = 'uniform', activation = 'relu'))
    # fifth hidden layer with 16 neurons and random uniform values for weights 
    classifier.add(Dense(units = 16, kernel_initializer = 'uniform', activation = 'relu'))
    # Output layer: we can use sigmoid function as we deal with [0,1] arrays for Y   
    classifier.add(Dense(units = 10, kernel_initializer = 'uniform', activation = 'softmax'))
    classifier.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
    return classifier
classifier = KerasClassifier(build_fn = build_classifier, epochs = 200)


#%% Train Test split
from sklearn.model_selection import train_test_split

test_size = 0.2
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=test_size)
# Reshape
X_train = X_train.reshape(-1, 100, 100,1)
X_test = X_test.reshape(-1, 100, 100, 1)

#ANN data preprocessing
number_of_train = X_train.shape[0]
number_of_test = X_test.shape[0]

#  we need to but X_train and X_test in 2D form for NN 
X_train_flatten = X_train.reshape(number_of_train,X_train.shape[1]*X_train.shape[2])
X_test_flatten = X_test .reshape(number_of_test,X_test.shape[1]*X_test.shape[2])
print("X train flatten",X_train_flatten.shape)
print("X test flatten",X_test_flatten.shape)


#evaluate and test ANN model
# cross validation and measurement of accuracy
train_accuracies = cross_val_score(estimator = classifier, X = X_train_flatten, y = Y_train, cv = 3)
test_accuracies = cross_val_score(estimator = classifier, X = X_test_flatten, y = Y_test, cv = 3)

train_mean = train_accuracies.mean()
test_mean = test_accuracies.mean()

print ("Second Model results - Softmax")
print("ANN train accuracy mean: "+ str(train_mean))
print("ANN test accuracy mean: "+ str(test_mean))

X train flatten (1647, 10000)
X test flatten (412, 10000)
Epoch 1/200
1098/1098 [==============================] - 0s 380us/step - loss: 2.3028 - accuracy: 0.0874
Epoch 2/200
1098/1098 [==============================] - 0s 198us/step - loss: 2.3027 - accuracy: 0.0956
Epoch 3/200
1098/1098 [==============================] - 0s 199us/step - loss: 2.3026 - accuracy: 0.1047
Epoch 4/200
1098/1098 [==============================] - 0s 182us/step - loss: 2.3026 - accuracy: 0.1047
Epoch 5/200
1098/1098 [==============================] - 0s 188us/step - loss: 2.3025 - accuracy: 0.1047
Epoch 6/200
1098/1098 [==============================] - 0s 158us/step - loss: 2.3025 - accuracy: 0.1047
Epoch 7/200
1098/1098 [==============================] - 0s 145us/step - loss: 2.3025 - accuracy: 0.1047
Epoch 8/200
1098/1098 [==============================] - 0s 145us/step - loss: 2.2980 - accuracy: 0.1047
Epoch 9/200
1098/1098 [==============================] - 0s 145us/step - loss: 2.2530 - accuracy: 0.14